In [1]:
from visual_behavior_glm.src.glm import GLM
import visual_behavior_glm.src.GLM_fit_tools as gft
import visual_behavior_glm.src.GLM_analysis_tools as gat
import visual_behavior_glm.src.GLM_visualization_tools as gvt

import visual_behavior.utilities as vbu
import visual_behavior.plotting as vbp
import visual_behavior.data_access.loading as loading
import visual_behavior.visualization.utils as vis_utils
import visual_behavior.database as db

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:

%matplotlib notebook
%widescreen

In [4]:
# oeid = 833629926 #slc example
oeid = 808621958 #vip example
version = 3
glm = GLM(oeid, version)

Fitting ophys_experiment_id: 808621958
Loading data


/home/dougo/Code/AllenSDK/allensdk/brain_observatory/sync_dataset.py:109: UserWarning: The loaded sync file contains the following deprecated line label keys: {'cam2_exposure', 'cam1_exposure'}. Consider updating the sync file line labels.
  self._check_line_labels()
/home/dougo/Code/AllenSDK/allensdk/brain_observatory/sync_dataset.py:109: UserWarning: The loaded sync file contains the following deprecated line label keys: {'cam2_exposure', 'cam1_exposure'}. Consider updating the sync file line labels.
  self._check_line_labels()


Processing df/f data
Build Design Matrix
    Adding kernel: intercept
    Adding kernel: time
    Adding kernel: pre_licks
    Adding kernel: post_licks
    Adding kernel: pre_lick_bouts
    Adding kernel: post_lick_bouts
    Adding kernel: rewards
    Adding kernel: change
    Adding kernel: hits
    Adding kernel: misses
    Adding kernel: false_alarms
    Adding kernel: correct_rejects
    Adding kernel: omissions
    Adding kernel: running
    Adding kernel: pupil


/home/dougo/Code/AllenSDK/allensdk/brain_observatory/sync_dataset.py:109: UserWarning: The loaded sync file contains the following deprecated line label keys: {'cam2_exposure', 'cam1_exposure'}. Consider updating the sync file line labels.
  self._check_line_labels()


    Adding kernel: image0
    Adding kernel: image1
    Adding kernel: image2
    Adding kernel: image3
    Adding kernel: image4
    Adding kernel: image5
    Adding kernel: image6
    Adding kernel: image7
    Adding kernel: model_bias
    Adding kernel: model_task0
    Adding kernel: model_omissions1
    Adding kernel: model_timing1D
Setting up CV
Evaluating Regularization values


    Fitting L2, 499.99999999999994: 100%|██████████| 5/5 [00:11<00:00,  2.21s/it]


Setting up model selection dropout
Iterating over model selection


    Fitting model, beh_model: 100%|██████████| 5/5 [00:05<00:00,  1.14s/it]


Saving results
Saving Design Matrix
Saving Events Table
Finished


In [ ]:
glm.results.sort_values(by='Full_avg_cv_var_test', ascending=False).head()

In [ ]:
len(glm.results)

In [ ]:
cell_specimen_id = glm.results.sort_values(by='Full_avg_cv_var_test', ascending=False).index[0]
glm.plot_dropout_summary(cell_specimen_id)

In [ ]:
movie = gvt.GLM_Movie(
    glm,
    cell_specimen_id = cell_specimen_id, 
    start_frame = 39400,
    end_frame = 39500
)
# ""
movie.make_cell_movie_frame(movie.ax, movie.glm, 20000, movie.cell_specimen_id, t_before=10, t_after=10)

In [ ]:
glm.plot_filters(cell_specimen_id)

In [ ]:
results = gft.L2_report(glm.fit)

In [ ]:
# fit['regularization'] = np.mean([fit['L2_grid'][x] for x in np.argmax(test_cv,1)])      
#         fit['L2_test_cv'] = test_cv
#         fit['L2_train_cv'] = train_cv
glm.fit['regularization']

In [ ]:
test_cv = glm.fit['L2_test_cv']

In [ ]:
len(np.argmax(test_cv,1))

In [ ]:
fit = glm.fit
test_cv = glm.fit['L2_test_cv']
lambdas = [fit['L2_grid'][x] for x in np.argmax(test_cv,1)]
fig,ax=plt.subplots()
ax.hist(lambdas,bins=np.arange(0,525,25))

In [ ]:
lam = 2
(lam * np.eye(glm.X.shape[-1])).shape

In [ ]:
licks = glm.session.dataset.licks
licks['time_to_last_lick'] = licks['timestamps'] - licks['timestamps'].shift()
licks['first_in_bout'] = licks['time_to_last_lick'] > 2
licks['first_in_bout'].value_counts()

In [ ]:
w_kernel = glm.W.loc[dict(weights=kernel_weight_names, cell_specimen_id=cell_specimen_id)]
w_kernel

In [ ]:
fig,ax=plt.subplots()
ax.plot(w_kernel)

In [ ]:
session = glm.session
session.dataset.trials.query('correct_reject')

In [ ]:
event = 'hit'
if event in ['hit','miss']:
    print('yes')